In [11]:
import os
import numpy as np
from PIL import Image
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
# 設置超參數
img_rows = 512
img_cols = 512
num_classes = 2
batch_size = 16
epochs = 20
lr = 1e-4
test_img="D:\\Calc\\Test_FULL"
test_mask="D:\\Calc\\Test_MASK"
train_img="D:\\Calc\\Train_FULL"
train_mask="D:\\Calc\\Train_MASK"

In [6]:
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras import models

def VGG19(img_input):
    # 512,512,3 -> 512,512,64
    x = layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='conv1-1')(img_input)
    x = layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='conv1-2')(x)
    conv1 = x
    
    # 512,512,64 -> 256,256,64
    x = layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), name='pool1')(x)
    
    # 256,256,64 -> 256,256,128
    x = layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='conv2-1')(x)
    x = layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='conv2-2')(x)
    conv2 = x
    
    # 256,256,128 -> 128,128,128
    x = layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), name='pool2')(x)
    
    # 128,128,128 -> 128,128,256
    x = layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='conv3-1')(x)
    x = layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='conv3-2')(x)
    x = layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='conv3-3')(x)
    x = layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='conv3-4')(x)
    conv3 = x
    
    # 128,128,256 -> 64,64,256
    x = layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), name='pool3')(x)
    
    # 64,64,256 -> 64,64,512
    x = layers.Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='conv4-1')(x)
    x = layers.Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='conv4-2')(x)
    x = layers.Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='conv4-3')(x)
    x = layers.Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='conv4-4')(x)
    conv4 = x
    
    # 64,64,512 -> 32,32,512
    x = layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), name='pool4')(x)
    
    # 32,32,512 -> 32,32,512
    x = layers.Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='conv5-1')(x)
    x = layers.Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='conv5-2')(x)
    x = layers.Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='conv5-3')(x)
    x = layers.Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='conv5-4')(x)
    conv5 = x
    
    return conv1, conv2, conv3, conv4, conv5

def Unet(input_shape=(512,512,3)):
    '''获取输入的图片的尺寸'''
    inputs = layers.Input(input_shape)
    
    '''
    获取之前VGG19生成的五个有效特征层
    conv1    512,512,64
    conv2    256,256,128
    conv3    128,128,256
    conv4    64,64,512
    conv5    32,32,512
    '''
    conv1, conv2, conv3, conv4, conv5 = VGG19(inputs)
    
    # 32,32,512 -> 64,64,512
    U5_up = layers.UpSampling2D(size=(2,2), interpolation='nearest')(conv5)
    
    # 64,64,512 + 64,64,512 -> 64,64,1024
    U4 = layers.Concatenate(axis=3)([conv4, U5_up])
    
    # 64,64,1024 -> 64,64,512
    U4 = layers.Conv2D(filters=512, kernel_size=3, strides=1, activation='relu', padding='same', kernel_initializer='he_normal')(U4)
    U4 = layers.Conv2D(filters=512, kernel_size=3, strides=1, activation='relu', padding='same', kernel_initializer='he_normal')(U4)
    
    # 64,64,512 -> 128,128,512
    U4_up = layers.UpSampling2D(size=(2,2), interpolation='nearest')(U4)
    
    # 128,128,512 + 128,128,256 -> 128,128,768
    U3 = layers.Concatenate(axis=3)([conv3, U4_up])
    
    # 128,128,768 -> 128,128,256
    U3 = layers.Conv2D(filters=256, kernel_size=3, strides=1, activation='relu', padding='same', kernel_initializer='he_normal')(U3)
    U3 = layers.Conv2D(filters=256, kernel_size=3, strides=1, activation='relu', padding='same', kernel_initializer='he_normal')(U3)
    
    # 128,128,256 -> 256,256,256
    U3_up = layers.UpSampling2D(size=(2,2), interpolation='nearest')(U3)
    
    # 256,256,256 + 256,256,128 -> 256,256,384
    U2 = layers.Concatenate(axis=3)([conv2, U3_up])
    
    # 256,256,384 -> 256,256,128
    U2 = layers.Conv2D(filters=128, kernel_size=3, strides=1, activation='relu', padding='same', kernel_initializer='he_normal')(U2)
    U2 = layers.Conv2D(filters=128, kernel_size=3, strides=1, activation='relu', padding='same', kernel_initializer='he_normal')(U2)
    
    # 256,256,128 -> 512,512,128
    U2_up = layers.UpSampling2D(size=(2,2), interpolation='nearest')(U2)
    
    # 512,512,128 + 512,512,64 -> 512,512,192
    U1 = layers.Concatenate(axis=3)([conv1, U2_up])
    
    # 512,512,192 -> 512,512,64
    U1 = layers.Conv2D(filters=64, kernel_size=3, strides=1, activation='relu', padding='same', kernel_initializer='he_normal')(U1)
    U1 = layers.Conv2D(filters=64, kernel_size=3, strides=1, activation='relu', padding='same', kernel_initializer='he_normal')(U1)
    
    # 512,512,64 -> 512,512,num_classes
    outputs = layers.Conv2D(filters=1, kernel_size=1, strides=1, activation='softmax')(U1)
    
    model = models.Model(inputs=inputs, outputs=outputs)
    return model


In [7]:
image_size = (512,512,3)
model = Unet(image_size)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1-1 (Conv2D)               (None, 512, 512, 64  1792        ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 conv1-2 (Conv2D)               (None, 512, 512, 64  36928       ['conv1-1[0][0]']                
                                )                                                           

 conv2d_13 (Conv2D)             (None, 256, 256, 12  442496      ['concatenate_6[0][0]']          
                                8)                                                                
                                                                                                  
 conv2d_14 (Conv2D)             (None, 256, 256, 12  147584      ['conv2d_13[0][0]']              
                                8)                                                                
                                                                                                  
 up_sampling2d_7 (UpSampling2D)  (None, 512, 512, 12  0          ['conv2d_14[0][0]']              
                                8)                                                                
                                                                                                  
 concatenate_7 (Concatenate)    (None, 512, 512, 19  0           ['conv1-2[0][0]',                
          

In [12]:
def load_train_data(train_img,train_mask):
    # Load images
    images = []
    for filename in os.listdir(train_img):
        img = cv2.imread(os.path.join(train_img, filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (img_rows, img_cols))
        images.append(img)

    # Load masks
    masks = []
    for filename in os.listdir(train_mask):
        mask = cv2.imread(os.path.join(train_mask, filename), cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask, (img_rows, img_cols))
        mask = np.expand_dims(mask, axis=-1)
        masks.append(mask)

    # Convert lists to numpy arrays
    images = np.array(images)
    masks = np.array(masks)

    return images, masks


def load_val_data(test_img,test_mask):
    # Load images
    images = []
    for filename in os.listdir(test_img):
        img = cv2.imread(os.path.join(test_img, filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (img_rows, img_cols))
        images.append(img)

    # Load masks
    masks = []
    for filename in os.listdir(test_mask):
        mask = cv2.imread(os.path.join(test_mask, filename), cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask, (img_rows, img_cols))
        mask = np.expand_dims(mask, axis=-1)
        masks.append(mask)

    # Convert lists to numpy arrays
    images = np.array(images)
    masks = np.array(masks)

    return images, masks
# Load train data
train_images, train_masks = load_train_data(train_img,train_mask)

# Load val data
val_images, val_masks = load_val_data(test_img,test_mask)


# model = create_vggunet(input_shape=(img_rows, img_cols, 3), img_channels=1)
optimizer = Adam(lr=lr)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# 定義回調函數
checkpoint = ModelCheckpoint(filepath='model.h5', monitor='val_loss', save_best_only=True, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, mode='min', verbose=1)

# callbacks_list = [model_checkpoint, early_stopping, reduce_lr]

# Fit the model
history = model.fit(train_images, train_masks, batch_size=batch_size, epochs=epochs,
                    validation_data=(val_images, val_masks), callbacks=[early_stopping, checkpoint,reduce_lr])

Epoch 1/20


ResourceExhaustedError: Graph execution error:

OOM when allocating tensor with shape[16,512,128,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node model_1/concatenate_5/concat-1-TransposeNHWCToNCHW-LayoutOptimizer}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_5990]

In [10]:
# 載入數據並進行預處理
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

train_image_generator = train_datagen.flow_from_directory(
    train_imgs,
    target_size=(img_rows, img_cols),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode=None,
    shuffle=True,
    seed=42)

train_mask_generator = train_datagen.flow_from_directory(
    train_mask,
    target_size=(img_rows, img_cols),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode=None,
    shuffle=True,
    seed=42)

val_image_generator = val_datagen.flow_from_directory(
    test_img,
    target_size=(img_rows, img_cols),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

val_mask_generator = val_datagen.flow_from_directory(
    test_mask,
    target_size=(img_rows, img_cols),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

train_generator = zip(train_image_generator, train_mask_generator)
val_generator = zip(val_image_generator, val_mask_generator)

# 創建模型
model = create_vggunet(input_shape=(img_rows, img_cols, 3), img_channels=num_classes)

# 編譯模型
optimizer = Adam(lr=lr)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# 定義回調函數
checkpoint = ModelCheckpoint(filepath='model.h5', monitor='val_loss', save_best_only=True, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

# 訓練模型
history = model.fit(train_generator,
                    steps_per_epoch=train_image_generator.samples//batch_size,
                    epochs=epochs,
                    validation_data=val_generator,
                    validation_steps=val_image_generator.samples//batch_size,
                    callbacks=[checkpoint, early_stopping])

# 評估模型
# model = load_model('model.h5')
# test_datagen = ImageDataGenerator(rescale=1./255)
# test_generator = test_datagen.flow_from_directory(
#     'test/',
#     target_size=(img_rows, img_cols),
#     color_mode='rgb',
#     batch_size=batch_size,
#     class_mode=None,
#     shuffle=False)
# test_image_names = test_generator.filenames
# probabilities = model.predict(test_generator)
# predictions = np.argmax(probabilities, axis=-1)
# test_labels = np.zeros(predictions.shape)
# test_labels[predictions==1] = 1
# test_labels =


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


ValueError: Graph disconnected: cannot obtain value for tensor KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'") at layer "block1_conv1". The following previous layers were accessed without issue: []